### Simple CNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
torch.cuda.is_available()

False

### Conv2d()설명

Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

• in_channels: 입력 채널 수을 뜻합니다. 흑백 이미지일 경우 1, RGB 값을 가진 이미지일 경우 3을 가진 경우가 많습니다.

• out_channels: 출력 채널 수을 뜻합니다.

• kernel_size: 커널 사이즈를 뜻합니다. int 혹은 tuple이 올 수 있습니다.

• stride: stride 사이즈를 뜻합니다. int 혹은 tuple이 올 수 있습니다. 기본 값은 1입니다.

• padding: padding 사이즈를 뜻합니다. int 혹은 tuple이 올 수 있습니다. 기본 값은 0입니다.

• padding_mode: padding mode를 설정할 수 있습니다. 기본 값은 'zeros' 입니다. 아직 zero padding만 지원 합니다.

• dilation: 커널 사이 간격 사이즈를 조절 합니다.

• groups: 입력 층의 그룹 수을 설정하여 입력의 채널 수를 그룹 수에 맞게 분류 합니다. 그 다음, 출력의 채널 수를 그룹 수에 맞게 분리하여, 입력 그룹과 출력 그룹의 짝을 지은 다음 해당 그룹 안에서만 연산이 이루어지게 합니다.

• bias: bias(편향) 값을 설정 할 지, 말지를 결정합니다. 기본 값은 True 입니다. Bias->예측값과 실제 정답과의 차이의 평균

In [3]:
class CNN(nn.Module) :
    def __init__(self) :
        super(CNN, self).__init__()
        # 이미지 특징(feature 등) 찾는 구간
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1) # in_channels=1은 grayscale의 이미지 채널 수를 뜻함
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
        
        # 나온 이미지의 자체 정보와 라벨 정보를 가지고 있는 구간
        self.fc1 = nn.Linear(10*12*12, 50) # (이미지 정보, 라벨 정보)
        self.fc2 = nn.Linear(50, 10) # 라벨 갯수를 수정해야 pre-trained model을 가져와서 사용할 수 있음. 당연히 호출 방법은 각 모델마다 다 다름
        
    def forward(self, x) :
        print("연산 전 x.size >>>", x.size())

        x = F.relu(self.conv1(x))
        print("conv1 연산 후 x.size >>>", x.size())

        x = F.relu(self.conv2(x))
        print("conv2 연산 후 x.size >>>", x.size())

        """차원 감소 후 --> x 값이 어떻게 변할까?"""
        # pytorch 기능 중 view(), reshape()은 숙지해둘 필요가 있다! --> tensor의 모양을 변경하는 기능
        # view()는 새로운 tensor의 모양을 변경 후 모양 유지
        # reshape()은 새로운 모양으로 변경 후 모양 유지 x, 원래의 모양을 유지하고 있음, copy를 먼저 해 둬야만 함
        # reshape()은 사용시 공간을 따로 만들어둠
        # reshape() 보다는 view()를 더 많이 사용함
        x = x.view(-1, 10 * 12 * 12) 
        print("차원 감소 후 >>>", x.size())
        
        x = F.relu(self.fc1(x))
        print("fc1 연산 후 x.size >>>", x.size())
        
        x = F.relu(self.fc2(x))
        print("fc2 연산 후 x.size >>>", x.size())

In [4]:
cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20)) # matrix size 20x20

연산 전 x.size >>> torch.Size([10, 1, 20, 20])
conv1 연산 후 x.size >>> torch.Size([10, 3, 16, 16])
conv2 연산 후 x.size >>> torch.Size([10, 10, 12, 12])
차원 감소 후 >>> torch.Size([10, 1440])
fc1 연산 후 x.size >>> torch.Size([10, 50])
fc2 연산 후 x.size >>> torch.Size([10, 10])
